In [1]:
from fastai.vision.all import *
import fastai
from wwf.vision.timm import *
import timm

In [2]:
import torch
torch.cuda.set_device(2)

In [3]:
path = Path('datasetCorregido/')
trainPath = path/'train'
testPath = path/'test'

In [4]:
dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=GrandparentSplitter(valid_name='test'),
                 get_y=parent_label,
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=256, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])

dlsTest = dbTest.dataloaders(path,bs=64)

In [5]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    EarlyStoppingCallback(patience=10),
    SaveModelCallback(fname='resnest50d'),
    MixUp()
]

In [14]:
learn = timm_learner(dlsTest, 'resnest50d_4s2x40d',cbs = callbacks,metrics=[accuracy,Precision(),Recall(),RocAucBinary()]).to_fp16()

In [23]:
learn.load('resnest50d_4')

In [24]:
learn.metrics=[accuracy,Precision(),Recall(),RocAucBinary()]

In [25]:
learn.validate()

Better model found at epoch 0 with valid_loss value: 0.5910780429840088.


(#4) [0.7203467488288879,0.5910780429840088,0.6381578947368421,0.6381578947368421]

In [26]:
preds,y=learn.get_preds()

In [27]:
predIdxs = np.argmax(preds.numpy(), axis=1)

In [28]:
from sklearn.metrics import confusion_matrix


In [29]:
print(confusion_matrix(predIdxs,y))

[[186 165]
 [165 291]]


In [30]:
tn, fp, fn, tp = confusion_matrix(predIdxs, y).ravel()

print(tn / (tn+fp))

0.5299145299145299


## TTA

In [31]:
preds,y=learn.tta(n=6)

In [32]:
predIdxs = np.argmax(preds.numpy(), axis=1)

In [33]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,roc_auc_score, classification_report


In [19]:
precision_score(predIdxs,y)

0.7653508771929824

In [20]:
recall_score(predIdxs,y)

0.6609848484848485

In [21]:
f1_score(predIdxs,y)

0.709349593495935

In [22]:
accuracy_score(predIdxs,y)

0.6456009913258984

In [23]:
roc_auc_score(y,([p[1] if not math.isnan(p[0]) else 0 for p in preds]))

0.6023641725396112

In [34]:
tn, fp, fn, tp = confusion_matrix(predIdxs, y).ravel()

print(tn / (tn+fp))

0.5636856368563685
